In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive
import glob
from matplotlib import pyplot as plt
import sys

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set directory for all images in the dataset
dataset_path = '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Datasets/AHE 5'

# Set if you want to display progress
display_images = False
verbose = True

# Get the directory for the image_0 and image_1 paths
image_0_paths = sorted(glob.glob(f'{dataset_path}/image_0/*.png'))
image_1_paths = sorted(glob.glob(f'{dataset_path}/image_1/*.png'))

# Check if the full dataset is picked up
if len(image_0_paths) != 456 or len(image_1_paths) != 456:
  print(f'ERROR - Full dataset not picked up\n\t{len(image_0_paths)=}\n\t{len(image_1_paths)=}\n')
  sys.exit()

# Initialize orb detector and matcher
orb = cv2.ORB_create()
bf = cv2.BFMatcher()

# Setting the treshold ratio for the ratio test
ratio_threshold = .6

# Initialize the number of good matches
num_good_matches = []

# Loop through all the images in the dataset
for i in range(len(image_0_paths)-2):

  # Get the current and next image_0 and image_1 image
  image_0_cur = cv2.imread(image_0_paths[i])
  image_1_cur = cv2.imread(image_1_paths[i])
  image_0_next = cv2.imread(image_0_paths[i+1])
  image_1_next = cv2.imread(image_1_paths[i+1])
    
  # Detect keypoints and descriptors
  image_0_cur_keypoints, image_0_cur_Descriptors = orb.detectAndCompute(image_0_cur, None)
  image_0_next_keypoints, image_0_next_Descriptors = orb.detectAndCompute(image_0_next, None)
  image_1_cur_keypoints, image_1_cur_Descriptors = orb.detectAndCompute(image_1_cur, None)
  image_1_next_keypoints, image_1_next_Descriptors = orb.detectAndCompute(image_1_next, None)

  # Get matches
  image_0_matches = bf.knnMatch(image_0_cur_Descriptors, image_0_next_Descriptors, k=2)
  image_1_matches = bf.knnMatch(image_1_cur_Descriptors, image_1_next_Descriptors, k=2)

  # Apply ratio test to keep only good matches
  image_0_good_matches = []
  image_1_good_matches = []
  for m,n in image_0_matches:
    if m.distance < ratio_threshold * n.distance:
      image_0_good_matches.append([m])
  for m,n in image_1_matches:
    if m.distance < ratio_threshold * n.distance:
      image_1_good_matches.append([m])
  
  # Keep track of the number of good matches
  num_good_matches.append( len(image_0_good_matches) + len(image_1_good_matches) )

  # Display index
  if verbose and i % 10 == 0:
    print(f'Index {i} ...')

  # Draw matches
  if display_images and i % 100 == 0:
    print(f'\nimage_0')
    image_0 = cv2.drawMatchesKnn(image_0_cur, image_0_cur_keypoints, image_0_next, image_0_next_keypoints, image_0_good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    image_0 = cv2.resize(image_0, (600,400))
    cv2_imshow(image_0)
    print('\nimage_1')
    image_1 = cv2.drawMatchesKnn(image_1_cur, image_1_cur_keypoints, image_1_next, image_1_next_keypoints, image_1_good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    image_1 = cv2.resize(image_1, (600,400))
    cv2_imshow(image_1)
    print()

# Analyze
mean_num_good_matches = np.mean(num_good_matches)
cov_num_good_matches = np.cov(num_good_matches)

# Create report
report = f'Number of good matches\n'
report += f'\t      Mean : {mean_num_good_matches}\n'
report += f'\tCovarience : {cov_num_good_matches}'

# Export report
with open(f'{dataset_path}/Orb Analysis Report.txt', 'w') as f:
  f.write(report)

# Print report
if verbose:
  print(f'\n\n{report}')

Index 0 ...
Index 10 ...
Index 20 ...
Index 30 ...
Index 40 ...
Index 50 ...
Index 60 ...
Index 70 ...
Index 80 ...
Index 90 ...
Index 100 ...
Index 110 ...
Index 120 ...
Index 130 ...
Index 140 ...
Index 150 ...
Index 160 ...
Index 170 ...
Index 180 ...
Index 190 ...
Index 200 ...
Index 210 ...
Index 220 ...
Index 230 ...
Index 240 ...
Index 250 ...
Index 260 ...
Index 270 ...
Index 280 ...
Index 290 ...
Index 300 ...
Index 310 ...
Index 320 ...
Index 330 ...
Index 340 ...
Index 350 ...
Index 360 ...
Index 370 ...
Index 380 ...
Index 390 ...
Index 400 ...
Index 410 ...
Index 420 ...
Index 430 ...
Index 440 ...
Index 450 ...


Number of good matches
	      Mean : 94.61453744493392
	Covarience : 2969.6546226332525
